In [11]:
import os
import openai
from interp_utils import reload_module
reload_module('noa_tools')
from noa_tools import threaded_query, query
from tokenizers import Tokenizer

import torch
from openai import OpenAI

client = OpenAI()


top_10k = torch.load('top_10k_indices.pt')
tokenizer = Tokenizer.from_pretrained('roneneldan/TinyStories-33M')

toks = [tokenizer.decode([tok_id]) for tok_id in top_10k]
toks = list(set([tok.lower()[1:] for tok in toks if tok[0] == ' ']))


template = '''Given the list of strings below, identify which strings match the following description:
{description}
To log the strings that match the description, call log_{title}. Copy strings exactly as they appear.

Here is the list of strings for you to process: {batch}'''


def format_nest(nest, **kwargs):
    if isinstance(nest, list):
        return [format_nest(item, **kwargs) for item in nest]
    elif isinstance(nest, dict):
        return {format_nest(k, **kwargs): format_nest(v, **kwargs) for k, v in nest.items()}
    elif isinstance(nest, str):
        return nest.format(**kwargs)
# def new_tok_label(title, description, examples=[], ):

functions = [
    {
    "name": "log_{title}",
    # "description": "",
    "parameters": {
        "type": "object",
        "properties": {
            "identified_{title}": {
                "type": "array",
                "description": "{description}",
                "items": {
                    "type": "string",
                }
            },
        },
    },
    "required": ["log_{title}"],
    },
]


format_nest(functions, title='adjectives', description='These are words that can be interpeted as adjectives in the right context.')

[{'name': 'log_adjectives',
  'parameters': {'type': 'object',
   'properties': {'identified_adjectives': {'type': 'array',
     'description': 'These are words that can be interpeted as adjectives in the right context.',
     'items': {'type': 'string'}}}},
  'required': ['log_adjectives']}]